In [ ]:
import json
import re

def time_to_seconds(time_str):
    """
    Convert a time string to seconds.
    Handles formats: hh:mm:ss, mm:ss, ss
    Now handles decimal numbers by rounding to nearest integer
    """
    try:
        # If it's just a number (seconds)
        if ':' not in time_str:
            # Round to nearest integer for decimal numbers
            return round(float(time_str))
        
        # Split the time string
        parts = time_str.split(':')
        
        if len(parts) == 3:  # hh:mm:ss
            h, m, s = parts
            # Round the seconds part if it contains decimals
            return int(h) * 3600 + int(m) * 60 + round(float(s))
        elif len(parts) == 2:  # mm:ss
            m, s = parts
            # Round the seconds part if it contains decimals
            return int(m) * 60 + round(float(s))
        else:
            return 0
    except (ValueError, TypeError):
        return 0


def validate_time_order(time_list):
    """
    Validate that the second time is greater than the first time.
    If not, return [0, 0].
    """
    if len(time_list) != 2:
        return [0, 0]
    
    if time_list[0] >= time_list[1]:
        return [0, 0]
    
    return time_list


def extract_time_values(text):
    """
    Extract time values from text and return them as a list of integers (seconds).
    """
    # First, try to find time patterns (hh:mm:ss or mm:ss)
    time_pattern = r'\d+:\d+(?::\d+)?'
    time_values = re.findall(time_pattern, text)
    
    # If not enough time values, look for decimal numbers in the text
    if len(time_values) < 2:
        # Look for patterns like "X.Y" or "X" where X and Y are digits
        # This will match both integer and decimal numbers
        decimal_pattern = r'\d+(?:\.\d+)?'
        
        # Find all matches in the text
        numbers = re.findall(decimal_pattern, text)
        
        # Filter out numbers that are part of time values
        numbers = [num for num in numbers if not any(num in tv for tv in time_values)]
        
        # Convert found decimal numbers to integers (rounded)
        time_values.extend(numbers)
    
    # Convert all values to seconds
    result = []
    for i in range(2):
        if i < len(time_values):
            result.append(time_to_seconds(time_values[i]))
        else:
            result.append(0)
    
    # Validate time order
    return validate_time_order(result)


def transform_json(input_json, pred_a_key):
    """
    Transform the input JSON by extracting time values from a_gt and a_pred
    and converting them to lists of integers (seconds).
    """
    output_json = []
    for item in input_json:
        transformed_item = item.copy()
        # Extract and convert times to seconds
        transformed_item['a_gt'] = extract_time_values(item['A'])
           
        transformed_item['a_pred'] = extract_time_values(item[pred_a_key])
        output_json.append(transformed_item)
    
    return output_json